In [1]:
from pathlib import Path
import pickle
import numpy as np
from tqdm import trange

# Directory creation for output
save_dir = Path("outputs")
save_dir.mkdir(exist_ok=True)

# Loading preprogrammed joint data
from flygym.preprogrammed import all_leg_dofs
from flygym.util import get_data_path
data_path = get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

# Simulation setup
run_time = 0.5
timestep = 1e-4
actuated_joints = all_leg_dofs
target_num_steps = int(run_time / timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * timestep

for i, joint in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[joint])

# Setting up the fly and camera for top view
from flygym import Fly, Camera, SingleFlySimulation
fly = Fly(spawn_pos=(0, 0, 0.5), actuated_joints=actuated_joints)
cam = Camera(fly=fly, camera_id="Animat/camera_top")

# Initializing simulation
sim = SingleFlySimulation(fly=fly, cameras=[cam])
obs, info = sim.reset()

# Running the simulation
for i in trange(target_num_steps):
    joint_pos = data_block[:, i]
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = sim.step(action)
    sim.render()

# Save the video from the top camera
cam.save_video(save_dir / "top_view_fly.mp4")

# Display the video
from IPython.display import Video
Video(str(save_dir / "top_view_fly.mp4"))


100%|██████████| 5000/5000 [00:33<00:00, 148.47it/s]


In [1]:
from pathlib import Path
import pickle
import numpy as np
from tqdm import trange

# Directory creation for output
save_dir = Path("outputs")
save_dir.mkdir(exist_ok=True)

# Loading preprogrammed joint data
from flygym.preprogrammed import all_leg_dofs
from flygym.util import get_data_path
data_path = get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

# Extend simulation setup to 10 seconds
run_time = 10.0  # Run for 10 seconds
timestep = 1e-4
actuated_joints = all_leg_dofs
target_num_steps = int(run_time / timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))

# Here, you might need to adjust how data is processed to ensure the fly moves straight.
# For simplicity, this example will use a constant set of joint positions.
# Assuming `data[joint]` provides a suitable posture for straight walking at some index `k`.
k = 0  # Index at which the fly is in a suitable straight-walking posture
for i, joint in enumerate(actuated_joints):
    data_block[i, :] = data[joint][k]  # Constant position for straight walking

# Setting up the fly and camera for top view
from flygym import Fly, Camera, SingleFlySimulation
fly = Fly(spawn_pos=(0, 0, 0.5), actuated_joints=actuated_joints)
cam = Camera(fly=fly, camera_id="Animat/camera_top")

# Initializing simulation
sim = SingleFlySimulation(fly=fly, cameras=[cam])
obs, info = sim.reset()

# Running the simulation
for i in trange(target_num_steps):
    joint_pos = data_block[:, i]
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = sim.step(action)
    sim.render()

# Save the video from the top camera
cam.save_video(save_dir / "top_view_fly.mp4")

# Display the video
from IPython.display import Video
Video(str(save_dir / "top_view_fly.mp4"))


100%|██████████| 100000/100000 [11:50<00:00, 140.77it/s]
